In [98]:
# Required packages

import numpy as np
import matplotlib.pyplot as plt

%run symplectic_euler.ipynb

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

plt.style.use('seaborn-poster')

In [99]:
class Hamiltonian:
    "Energy preserved neural network"
    def __init__(self, solver, input_dim, output_dim, hidden_layer, each_neuron, activation_func, epoch,
                     batch_size, learning_rate):
        
        self.z, self.t, self.H, self.initial_point, self.h = solver
        
        # Neural Network Model with MSE Loss
        self.model = self.NN(inputs =input_dim, output=output_dim, hidden_layer=hidden_layer, neuron=each_neuron,
                                     activation=activation_func)
        self.model_train = self.train(model=self.model, inputs=self.t, target=self.z, learning_rate=learning_rate,
                                    batch_size=batch_size, epochs=epoch)
        
        #Neural Network Model with Energy Preserved Loss
        
        self.energy_model = self.NN(inputs= input_dim,output=output_dim, hidden_layer=hidden_layer, neuron=each_neuron,
                                        activation=activation_func)
        
        # Parametreleri atama ve veri arttırma
        self.energy_model.set_weights(self.model_train.get_weights())
        t_created = np.arange(self.initial_point[0], self.initial_point, self.h)
        target_created = self.model_train(t_created)
        
        self.energy_model_train = self.train(model=self.energy_model, inputs=t_created, target=target_created,
                                             learning_rate=learning_rate, loss=self.custom_loss, batch_size=batch_size,
                                             epochs=epoch)
    
    
    # Neural network with MSE loss
    def NN(self, inputs, output, hidden_layer, neuron, activation="relu"):

        inp = Input(shape=(inputs,), name= "Input_Layer")
        for i in range(0, hidden_layer):
            if i == 0:
                x = Dense(neuron, activation=activation, name="Hidden_Layer_{}".format(i+1))(inp)
            else:
                x = Dense(neuron, activation=activation, name="Hidden_Layer_{}".format(i+1))(x)
        x = Dense(output, name="Output_Layer")(x)

        out = tf.constant([[self.initial_point[0],self.initial_point[1]]],
                              dtype=tf.float32) + (1-tf.math.exp(-inp**2)) * x

        return Model(inputs=inp, outputs=out)

    # Neural Network Model Compile and Fit
    def train(self, model, inputs, target, learning_rate=0.01, loss="mse", batch_size=32, epochs=1000):
        
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(optimizer=opt, loss=loss)
        model.fit(x=inputs, y=target, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=True)
        
        return model

    def custom_loss(self, y_true, y_pred):
        return tf.keras.losses.MSE(0.5, self.H(y_pred[None,:,0], y_pred[None, :,1])) +  0.5 * tf.keras.losses.MSE(y_true,y_pred)


In [100]:
def T(p):
    return (p**2)/2

def V(q):
    return (q**2)/2

Hamiltonian(symplectic_euler(T, V, initial_point=[0.,1.], t=[0,2*np.pi], h=0.5), input_dim=1, output_dim=2,
            hidden_layer=20, each_neuron=32, activation_func="sigmoid", epoch=1000 ,batch_size=1000000, learning_rate=0.001)

Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 1.4288
Epoch 2/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.3694
Epoch 3/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.3120
Epoch 4/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.2565
Epoch 5/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.2030
Epoch 6/1000
1/1 [==============================] - 0s 11ms/step - loss: 1.1514
Epoch 7/1000
1/1 [==============================] - 0s 12ms/step - loss: 1.1019
Epoch 8/1000
1/1 [==============================] - 0s 12ms/step - loss: 1.0544
Epoch 9/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.0089
Epoch 10/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.9654
Epoch 11/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.9240
Epoch 12/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.8847
Epoch 13/1000
1/1 [======================

1/1 [==============================] - 0s 7ms/step - loss: 0.4599
Epoch 204/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 205/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.4599
Epoch 206/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 207/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 208/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 209/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 210/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 211/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 212/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 213/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 214/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.4599
Epoch 215/1000
1/1 [=======================

1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 305/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.4599
Epoch 306/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.4599
Epoch 307/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 308/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 309/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 310/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 311/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 312/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 313/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 314/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.4599
Epoch 315/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 316/1000
1/1 [=====================

Epoch 405/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.4599
Epoch 406/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 407/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.4599
Epoch 408/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 409/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 410/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 411/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 412/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.4599
Epoch 413/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.4599
Epoch 414/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 415/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 416/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 417/1000
1/1 [======

1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 607/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 608/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 609/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.4599
Epoch 610/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 611/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 612/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 613/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 614/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.4599
Epoch 615/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.4599
Epoch 616/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.4599
Epoch 617/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 618/1000
1/1 [====================

1/1 [==============================] - 0s 11ms/step - loss: 0.4599
Epoch 708/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 709/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.4599
Epoch 710/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 711/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 712/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 713/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 714/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 715/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.4599
Epoch 716/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 717/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 718/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 719/1000
1/1 [======================

1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 910/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.4599
Epoch 911/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.4599
Epoch 912/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 913/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 914/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 915/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 916/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 917/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.4599
Epoch 918/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4599
Epoch 919/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.4599
Epoch 920/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.4599
Epoch 921/1000
1/1 [=======================

Epoch 110/1000
1/1 [==============================] - 1s 653ms/step - loss: 0.1328
Epoch 111/1000
1/1 [==============================] - 1s 654ms/step - loss: 0.1328
Epoch 112/1000
1/1 [==============================] - 1s 753ms/step - loss: 0.1328
Epoch 113/1000
1/1 [==============================] - 1s 725ms/step - loss: 0.1328
Epoch 114/1000
1/1 [==============================] - 1s 747ms/step - loss: 0.1328
Epoch 115/1000
1/1 [==============================] - 1s 686ms/step - loss: 0.1328
Epoch 116/1000
1/1 [==============================] - 1s 741ms/step - loss: 0.1328
Epoch 117/1000
1/1 [==============================] - 1s 765ms/step - loss: 0.1328
Epoch 118/1000
1/1 [==============================] - 1s 669ms/step - loss: 0.1328
Epoch 119/1000
1/1 [==============================] - 1s 707ms/step - loss: 0.1328
Epoch 120/1000
1/1 [==============================] - 1s 723ms/step - loss: 0.1328
Epoch 121/1000
1/1 [==============================] - 1s 758ms/step - loss: 0.1328
Epoc

KeyboardInterrupt: 

In [ ]:
Hamiltonian.input_dim